In [ ]:
import pandas as pd

df1 = pd.read_csv("electronics_sample 3.csv")
df2 = pd.read_csv("neutral_reviews_5000 1.csv")
df3 = pd.read_csv("synthetic_reviews 1.csv")
df4 = pd.read_csv("synthetic_reviews_new 1.csv")

df = pd.concat([df1, df2, df3, df4], ignore_index=True)
display(df)

,overall,vote,reviewTime,reviewText,summary,sentiment
0,2.0,0.0,2010-02-10,Tech support is the worst,1265760000,NaN
1,2.0,0.0,2016-10-24,Screws were missing from the bracket and beaut...,Spend a little more and get much better.,NaN
2,1.0,0.0,2017-07-10,Trouble connecting and staying connected via b...,1499644800,NaN
3,4.0,5.0,2013-05-02,I purchased this unit for our RV to replace an...,Receiver Offers a Lot of Flexibility & Complexity,NaN
4,3.0,0.0,2013-01-04,It works. Nuff said but the review requires 1...,It's a cable,NaN
...,...,...,...,...,...,...
170529,NaN,NaN,NaN,Absolutely loved this product! Highly recommen...,NaN,2.0
170530,NaN,NaN,NaN,"It's an okay product, nothing special.",NaN,1.0
170531,NaN,NaN,NaN,"Not impressed, but not disappointed either.",NaN,1.0
170532,NaN,NaN,NaN,Absolutely loved this product! Highly recommen...,NaN,2.0


In [ ]:
print(df["sentiment"].isna().sum())

def assign_sentiment(row):
    if pd.isna(row["sentiment"]):
        if row["overall"] in [1.0, 2.0]:
            return 0
        elif row["overall"] == 3.0:
            return 1
        elif row["overall"] in [4.0, 5.0]:
            return 2
    return row["sentiment"]  #

df["sentiment"] = df.apply(assign_sentiment, axis=1)


print(df["sentiment"].isna().sum())
display(df)


15534
0


,overall,vote,reviewTime,reviewText,summary,sentiment
0,2.0,0.0,2010-02-10,Tech support is the worst,1265760000,0.0
1,2.0,0.0,2016-10-24,Screws were missing from the bracket and beaut...,Spend a little more and get much better.,0.0
2,1.0,0.0,2017-07-10,Trouble connecting and staying connected via b...,1499644800,0.0
3,4.0,5.0,2013-05-02,I purchased this unit for our RV to replace an...,Receiver Offers a Lot of Flexibility & Complexity,2.0
4,3.0,0.0,2013-01-04,It works. Nuff said but the review requires 1...,It's a cable,1.0
...,...,...,...,...,...,...
170529,NaN,NaN,NaN,Absolutely loved this product! Highly recommen...,NaN,2.0
170530,NaN,NaN,NaN,"It's an okay product, nothing special.",NaN,1.0
170531,NaN,NaN,NaN,"Not impressed, but not disappointed either.",NaN,1.0
170532,NaN,NaN,NaN,Absolutely loved this product! Highly recommen...,NaN,2.0


In [ ]:
df = df[["reviewText", "sentiment"]]
df = df.dropna()
display(df)

,reviewText,sentiment
0,Tech support is the worst,0.0
1,Screws were missing from the bracket and beaut...,0.0
2,Trouble connecting and staying connected via b...,0.0
3,I purchased this unit for our RV to replace an...,2.0
4,It works. Nuff said but the review requires 1...,1.0
...,...,...
170529,Absolutely loved this product! Highly recommen...,2.0
170530,"It's an okay product, nothing special.",1.0
170531,"Not impressed, but not disappointed either.",1.0
170532,Absolutely loved this product! Highly recommen...,2.0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Convert text into numerical format (TF-IDF)
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)

# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    df["reviewText"], df["sentiment"], test_size=0.2, random_state=42
)

# Fit and transform data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Train logistic regression model
model = LogisticRegression(max_iter=300)
model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

# Save model and vectorizer
joblib.dump(model, "sentiment_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

Accuracy: 0.9635851877913625
              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96      9268
         1.0       0.98      0.96      0.97     13629
         2.0       0.95      0.97      0.96     11210

    accuracy                           0.96     34107
   macro avg       0.96      0.96      0.96     34107
weighted avg       0.96      0.96      0.96     34107



['tfidf_vectorizer.pkl']

In [ ]:
import joblib

# Load trained model and vectorizer
model = joblib.load("sentiment_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Take user input
review = input("Enter your review: ")

# Convert input to TF-IDF
review_tfidf = vectorizer.transform([review])

# Predict sentiment
prediction = model.predict(review_tfidf)[0]

# Map predictions to labels
sentiment_label = {0: "Negative", 1: "Neutral", 2: "Positive"}
print("Predicted Sentiment:", sentiment_label[prediction])


Enter your review: Average product. Does the job
Predicted Sentiment: Neutral
